# Assignment 2 : SMS Spam Classification
# Rishika Tibrewal, MDS202135

## Importing Libraries

In [ ]:
! pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 KB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8

In [ ]:
! pip install mlflow --quiet

In [ ]:
! pip install pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mlflow

from pyngrok import ngrok
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,precision_recall_curve,auc
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

## Loading training, validation, test data

In [ ]:
train=pd.read_csv("Training Data.csv")
val=pd.read_csv("Validation Data.csv")
test=pd.read_csv("Test Data.csv")

In [ ]:
train.head()

,Message,Label
0,send yetty number pls,0
1,1 000 cash 2 000 prize claim call09050000327,1
2,hey babe going ever figure going new year,0
3,let use next week princess,0
4,nice talking please dont forget pix want see,0


In [ ]:
val.head()

,Message,Label
0,da happy sit together na,0
1,2nd attempt contract u week top prize either 1...,1
2,text carlos let know hang,0
3,shant disturb u anymore jia,0
4,hello u u 2 u b moving flat co need arrange pi...,0


In [ ]:
test.head()

,Message,Label
0,watching tv lor funny bluff 4 wat izzit thk im...,0
1,well done 4 costa del sol holiday 5000 await c...,1
2,sm ac sptv new jersey devil detroit red wing p...,1
3,love working home,0
4,get ready put excellent sub face,0


In [ ]:
y_train,X_train=train["Label"],train["Message"]
y_val,X_val=val["Label"],val["Message"]
y_test,X_test=test["Label"],test["Message"]

## Converting string to vectors, to give as input to the models

In [ ]:
# replacing NAN entries by empty string
X_train = X_train.replace(np.nan, '', regex=True)
X_val = X_val.replace(np.nan, '', regex=True)
X_test = X_test.replace(np.nan, '', regex=True)

In [ ]:
count = CountVectorizer().fit(X_train)
X_train = count.transform(X_train)
X_val = count.transform(X_val)
X_test = count.transform(X_test)

In [ ]:
tfidf_transformer = TfidfTransformer()
tfidf_train = tfidf_transformer.fit_transform(X_train)
tfidf_val = tfidf_transformer.fit_transform(X_val)
tfidf_test = tfidf_transformer.fit_transform(X_test)

In [ ]:
tfidf_train.shape,tfidf_val.shape,tfidf_test.shape

((4025, 6778), (711, 6778), (836, 6778))

## Training Models

In [ ]:
def eval_metrics(actual, pred):
    precision, recall, thresholds = precision_recall_curve(actual, pred)
    auc_precision_recall = auc(recall, precision)
    return (auc_precision_recall)

In [ ]:
mlflow.sklearn.autolog()

n_estimators = np.arange(200,501,100)
max_depth = np.arange(5,8,1)

for i in range(4):
  for j in range(3):
    clf = RandomForestClassifier(n_estimators = n_estimators[i], max_depth = max_depth[j],random_state=101)
    clf.fit(tfidf_train, y_train)

    y_pred = clf.predict(tfidf_test)
    aucpr = eval_metrics(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)

    with mlflow.start_run(run_name=f"n_estimators : {n_estimators[i]}, max_depth : {max_depth[j]}"):
      
        mlflow.log_param("n_estimators", n_estimators[i])
        mlflow.log_param("max_depth", max_depth[j])
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("AUCPR",aucpr)
        mlflow.sklearn.log_model(clf, "model")

        print("\nRandom Classifier Model (no_of_estimator={:f}, max_depth={:f}):".format(n_estimators[i], max_depth[j]))
        print(f"Accuracy: {acc}")
        print(f"AUCPR: {aucpr} \n\n")
        

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.sklearn.log_model(
            sk_model=clf,
            artifact_path="sklearn-model",
            registered_model_name="random-forest-classification-model"
        )
        if tracking_url_type_store != "file":
          mlflow.sklearn.log_model(clf, "model", registered_model_name="Randomclassifier")
        else:
          mlflow.sklearn.log_model(clf, "model")


2023/02/25 10:56:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '648555b70d9044f38994692392aee02c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Random Classifier Model (no_of_estimator=200.000000, max_depth=5.000000):
Accuracy: 0.8755980861244019
AUCPR: 0.562200956937799 




Successfully registered model 'random-forest-classification-model'.
2023/02/25 10:56:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-classification-model, version 1
Created version '1' of model 'random-forest-classification-model'.
2023/02/25 10:56:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e6bad2912b3a41a8a0531155e0e30747', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Random Classifier Model (no_of_estimator=200.000000, max_depth=6.000000):
Accuracy: 0.8767942583732058
AUCPR: 0.5664105631210894 




Registered model 'random-forest-classification-model' already exists. Creating a new version of this model...
2023/02/25 10:56:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-classification-model, version 2
Created version '2' of model 'random-forest-classification-model'.
2023/02/25 10:57:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1467e0f3591d4fbb8164721587f8271e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


...

Registered model 'random-forest-classification-model' already exists. Creating a new version of this model...
2023/02/25 10:58:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-classification-model, version 10
Created version '10' of model 'random-forest-classification-model'.
2023/02/25 10:58:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a54c911775924e7c98ce409d3f968304', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Random Classifier Model (no_of_estimator=500.000000, max_depth=6.000000):
Accuracy: 0.8779904306220095
AUCPR: 0.5706201693043799 




Registered model 'random-forest-classification-model' already exists. Creating a new version of this model...
2023/02/25 10:59:11 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-classification-model, version 11
Created version '11' of model 'random-forest-classification-model'.
2023/02/25 10:59:13 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0be0876170cf482c81a1473b3e6552b6', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Random Classifier Model (no_of_estimator=500.000000, max_depth=7.000000):
Accuracy: 0.8803827751196173
AUCPR: 0.5790393816709606 




Registered model 'random-forest-classification-model' already exists. Creating a new version of this model...
2023/02/25 10:59:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-classification-model, version 12
Created version '12' of model 'random-forest-classification-model'.


In [ ]:
for i in range(1,13):
  print("\n",mlflow.tracking.MlflowClient().get_model_version("random-forest-classification-model",str(i)))


 <ModelVersion: creation_timestamp=1677322604670, current_stage='None', description=None, last_updated_timestamp=1677322604670, name='random-forest-classification-model', run_id='439e18e9e51d434e81f74a0d75c4362f', run_link=None, source='file:///content/mlruns/0/439e18e9e51d434e81f74a0d75c4362f/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>

 <ModelVersion: creation_timestamp=1677322619446, current_stage='None', description=None, last_updated_timestamp=1677322619446, name='random-forest-classification-model', run_id='526db7f1a38b41b298e880d351cac29d', run_link=None, source='file:///content/mlruns/0/526db7f1a38b41b298e880d351cac29d/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=2>

 <ModelVersion: creation_timestamp=1677322631443, current_stage='None', description=None, last_updated_timestamp=1677322631443, name='random-forest-classification-model', run_id='2362e03095b7439e8f9b30a631f6b8f2',

In [ ]:
get_ipython().system_raw("mlflow ui --port 5000 &")
ngrok.kill()
ng_token="2ME7aHUHn9pViBPPQmUNA8NVloG_5YLtL8jkMAbSizJ7b9PVS"
ngrok.set_auth_token(ng_token)
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("Track URL: ",ngrok_tunnel.public_url)

Track URL:  https://8ac0-35-201-210-190.ngrok.io


### Model 2: Multinomial Naive Bayes

In [ ]:
mlflow.sklearn.autolog()

clf = MultinomialNB()
clf.fit(tfidf_train, y_train)

y_pred = clf.predict(tfidf_test)
aucpr = eval_metrics(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

with mlflow.start_run(run_name="Multinomial Naive Bayes"):

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("AUCPR",aucpr)
    mlflow.sklearn.log_model(clf, "model")

    print("\nMultinomial Naive Bayes")
    print(f"Accuracy: {acc}")
    print(f"AUCPR: {aucpr} \n")
    

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="sklearn-model",
        registered_model_name="multinomial-nb-model"
    )
    if tracking_url_type_store != "file":
      mlflow.sklearn.log_model(clf, "model", registered_model_name="multinomial-nb-model")
    else:
      mlflow.sklearn.log_model(clf, "model")


2023/02/25 11:00:17 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1c62b642847c4487898ce41a5969f9a5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Multinomial Naive Bayes
Accuracy: 0.9617224880382775
AUCPR: 0.8652926021347074 



Successfully registered model 'multinomial-nb-model'.
2023/02/25 11:00:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: multinomial-nb-model, version 1
Created version '1' of model 'multinomial-nb-model'.


In [ ]:
print(mlflow.tracking.MlflowClient().get_model_version("multinomial-nb-model", '1'))

<ModelVersion: creation_timestamp=1677322825926, current_stage='None', description=None, last_updated_timestamp=1677322825926, name='multinomial-nb-model', run_id='71254120dac14a24b6b9de6b5d1ca3ad', run_link=None, source='file:///content/mlruns/0/71254120dac14a24b6b9de6b5d1ca3ad/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>


Model 3: MLP Classifier

In [ ]:
mlflow.sklearn.autolog()

clf = MLPClassifier(random_state=101,learning_rate='adaptive')
clf.fit(tfidf_train, y_train)

y_pred = clf.predict(tfidf_test)
aucpr = eval_metrics(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test,y_pred)
t_n, f_p, f_n, t_p = conf_matrix.ravel()

with mlflow.start_run(run_name="Multilayer Perceptron"):

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("AUCPR",aucpr)
    mlflow.log_metric("True Negative", t_n)
    mlflow.log_metric("False Positive", f_p)
    mlflow.log_metric("False Negative", f_n)
    mlflow.log_metric("True Positive", t_p)
    mlflow.sklearn.log_model(clf, "model")

    print("\nMultilayer Perceptron")
    print(f"Accuracy: {acc}")
    print(f"AUCPR: {aucpr} \n")
    

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="sklearn-model",
        registered_model_name="multilayer-perceptron-model"
    )
    if tracking_url_type_store != "file":
      mlflow.sklearn.log_model(clf, "model", registered_model_name="multilayer-perceptron-model")
    else:
      mlflow.sklearn.log_model(clf, "model")


2023/02/25 11:00:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '278581571ec04aec9517ac1988cde769', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Multilayer Perceptron
Accuracy: 0.9844497607655502
AUCPR: 0.943168016194332 



Successfully registered model 'multilayer-perceptron-model'.
2023/02/25 11:01:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: multilayer-perceptron-model, version 1
Created version '1' of model 'multilayer-perceptron-model'.


In [ ]:
print(mlflow.tracking.MlflowClient().get_model_version("multilayer-perceptron-model", '1'))